<a href="https://colab.research.google.com/github/AOdrio/Datathon/blob/main/C%C3%B3digo_en_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(ggplot2)
#install.packages('dplyr')
library(dplyr)
#install.packages('MASS')
library(MASS)
install.packages('randomForest')
library(randomForest)
install.packages('caret')
library("caret")
library("googledrive")
install.packages("mice")
library(mice)


In [2]:
id <- "1KH1fB83JDAWcMRCHKFf9SmcGH7MQkupE" # google file ID
df <- read.csv(sprintf("https://docs.google.com/uc?id=%s&export=download", id))
head(df)

,X,vigente,vigenteAvg3M,vencido,vencidoAvg3M,avgcontingenciaU1M,avgcontingenciaU3M,sumcontingenciaU1M,sumcontingenciaU3M,maxcontingenciaU1M,⋯,numerobancosprivados,numerobancospublicos,numerofinancieras,numerobancos,numerode5ult18meses,numerode2Bult18meses,numerode3ult18meses,edad,numeromesesreportados,ingreso
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,1,4282.155,4679.336,0.000,0.000,4923.124,3918.527,19692.50,18439.94,6653.07,⋯,1,0,3,1,0,0,0,63,24,31888.47
2,2,9793.123,9450.927,83970.243,91126.581,20809.053,19126.816,41618.11,49280.50,30881.25,⋯,0,1,1,1,4,0,2,67,24,62341.14
3,3,531419.702,512057.085,0.000,0.000,36813.149,46144.419,36813.15,36789.29,36813.15,⋯,2,1,0,3,0,0,0,NA,24,100599.76
4,4,0.000,0.000,2266.729,2633.112,11533.815,10488.339,23067.63,24025.95,19044.72,⋯,4,0,0,4,4,2,3,NA,24,46102.90
5,5,112394.057,110856.021,0.000,0.000,41131.334,42820.024,82262.67,77614.23,43944.58,⋯,4,0,0,4,0,0,0,65,14,109419.22
6,6,0.000,0.000,15136.639,15028.350,38851.830,36968.398,38851.83,37783.35,38851.83,⋯,0,0,3,0,9,0,1,NA,24,59375.89


In [3]:
str(df)

'data.frame':	40000 obs. of  25 variables:
 $ X                    : int  1 2 3 4 5 6 7 8 9 10 ...
 $ vigente              : num  4282 9793 531420 0 112394 ...
 $ vigenteAvg3M         : num  4679 9451 512057 0 110856 ...
 $ vencido              : num  0 83970 0 2267 0 ...
 $ vencidoAvg3M         : num  0 91127 0 2633 0 ...
 $ avgcontingenciaU1M   : num  4923 20809 36813 11534 41131 ...
 $ avgcontingenciaU3M   : num  3919 19127 46144 10488 42820 ...
 $ sumcontingenciaU1M   : num  19692 41618 36813 23068 82263 ...
 $ sumcontingenciaU3M   : num  18440 49280 36789 24026 77614 ...
 $ maxcontingenciaU1M   : num  6653 30881 36813 19045 43945 ...
 $ maxcontingenciaU3M   : num  7996 32794 46144 17864 42820 ...
 $ numcontingenciaU1M   : int  4 2 1 2 2 1 0 2 4 0 ...
 $ garantiacomputable   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ peorcalificacionBCU  : chr  "1C" "3" "1C" "5" ...
 $ numeroinstituciones  : int  4 2 3 4 4 3 2 6 4 1 ...
 $ numerobancosprivados : int  1 0 2 4 4 0 0 0 0 0 ...
 $ numerobancosp

In [4]:
#Convertir variables cualitativas

#unique(df$peorcalificacionBCU)
df$peorcalificacionBCU = ifelse(df$peorcalificacionBCU == '1C', 1,
ifelse(df$peorcalificacionBCU == '2A',2, ifelse(df$peorcalificacionBCU == '2B',3,
ifelse(df$peorcalificacionBCU == '3',4, ifelse(df$peorcalificacionBCU == '4',5,6)))))

unique(df$peorcalificacionBCU)

[1] 1 4 6 3 5 2

In [5]:
# Transformación de las variables categóricas en factor

df <- transform(df, peorcalificacionBCU = as.factor(peorcalificacionBCU))

str(df)

'data.frame':	40000 obs. of  25 variables:
 $ X                    : int  1 2 3 4 5 6 7 8 9 10 ...
 $ vigente              : num  4282 9793 531420 0 112394 ...
 $ vigenteAvg3M         : num  4679 9451 512057 0 110856 ...
 $ vencido              : num  0 83970 0 2267 0 ...
 $ vencidoAvg3M         : num  0 91127 0 2633 0 ...
 $ avgcontingenciaU1M   : num  4923 20809 36813 11534 41131 ...
 $ avgcontingenciaU3M   : num  3919 19127 46144 10488 42820 ...
 $ sumcontingenciaU1M   : num  19692 41618 36813 23068 82263 ...
 $ sumcontingenciaU3M   : num  18440 49280 36789 24026 77614 ...
 $ maxcontingenciaU1M   : num  6653 30881 36813 19045 43945 ...
 $ maxcontingenciaU3M   : num  7996 32794 46144 17864 42820 ...
 $ numcontingenciaU1M   : int  4 2 1 2 2 1 0 2 4 0 ...
 $ garantiacomputable   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ peorcalificacionBCU  : Factor w/ 6 levels "1","2","3","4",..: 1 4 1 6 1 4 1 1 1 6 ...
 $ numeroinstituciones  : int  4 2 3 4 4 3 2 6 4 1 ...
 $ numerobancosprivados : int  1 0 

In [6]:
# Evaluando Missing Values

na_count <-sapply(df, function(y) sum(length(which(is.na(y)))))
na_count <- data.frame(na_count)
na_count

,na_count
,<int>
X,0
vigente,0
vigenteAvg3M,0
vencido,0
vencidoAvg3M,0
avgcontingenciaU1M,0
avgcontingenciaU3M,0
sumcontingenciaU1M,0
sumcontingenciaU3M,0


In [7]:
# Completando Missing Values

mice_imputes = mice(df, m=5, maxit = 50)
#methods(mice)

df = complete(mice_imputes,5)

na_count <-sapply(df, function(y) sum(length(which(is.na(y)))))
na_count <- data.frame(na_count)
na_count


 iter imp variable
  1   1  edad
  1   2  edad
  1   3  edad
  1   4  edad
  1   5  edad
  2   1  edad
  2   2  edad
  2   3  edad
  2   4  edad
  2   5  edad
  3   1  edad
  3   2  edad
  3   3  edad
  3   4  edad
  3   5  edad
  4   1  edad
  4   2  edad
  4   3  edad
  4   4  edad
  4   5  edad
  5   1  edad
  5   2  edad
  5   3  edad
  5   4  edad
  5   5  edad
  6   1  edad
  6   2  edad
  6   3  edad
  6   4  edad
  6   5  edad
  7   1  edad
  7   2  edad
  7   3  edad
  7   4  edad
  7   5  edad
  8   1  edad
  8   2  edad
  8   3  edad
  8   4  edad
  8   5  edad
  9   1  edad
  9   2  edad
  9   3  edad
  9   4  edad
  9   5  edad
  10   1  edad
  10   2  edad
  10   3  edad
  10   4  edad
  10   5  edad
  11   1  edad
  11   2  edad
  11   3  edad
  11   4  edad
  11   5  edad
  12   1  edad
  12   2  edad
  12   3  edad
  12   4  edad
  12   5  edad
  13   1  edad
  13   2  edad
  13   3  edad
  13   4  edad
  13   5  edad
  14   1  edad
  14   2  edad
  14   3  edad
  14 

Warning message:
“Number of logged events: 250”


,na_count
,<int>
X,0
vigente,0
vigenteAvg3M,0
vencido,0
vencidoAvg3M,0
avgcontingenciaU1M,0
avgcontingenciaU3M,0
sumcontingenciaU1M,0
sumcontingenciaU3M,0


In [8]:
head(df)

,X,vigente,vigenteAvg3M,vencido,vencidoAvg3M,avgcontingenciaU1M,avgcontingenciaU3M,sumcontingenciaU1M,sumcontingenciaU3M,maxcontingenciaU1M,⋯,numerobancosprivados,numerobancospublicos,numerofinancieras,numerobancos,numerode5ult18meses,numerode2Bult18meses,numerode3ult18meses,edad,numeromesesreportados,ingreso
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,1,4282.155,4679.336,0.000,0.000,4923.124,3918.527,19692.50,18439.94,6653.07,⋯,1,0,3,1,0,0,0,63,24,31888.47
2,2,9793.123,9450.927,83970.243,91126.581,20809.053,19126.816,41618.11,49280.50,30881.25,⋯,0,1,1,1,4,0,2,67,24,62341.14
3,3,531419.702,512057.085,0.000,0.000,36813.149,46144.419,36813.15,36789.29,36813.15,⋯,2,1,0,3,0,0,0,66,24,100599.76
4,4,0.000,0.000,2266.729,2633.112,11533.815,10488.339,23067.63,24025.95,19044.72,⋯,4,0,0,4,4,2,3,36,24,46102.90
5,5,112394.057,110856.021,0.000,0.000,41131.334,42820.024,82262.67,77614.23,43944.58,⋯,4,0,0,4,0,0,0,65,14,109419.22
6,6,0.000,0.000,15136.639,15028.350,38851.830,36968.398,38851.83,37783.35,38851.83,⋯,0,0,3,0,9,0,1,48,24,59375.89


In [9]:
# Partición del dataset entre train (80%) y test (20%)

set.seed(27)
df_index = createDataPartition(df$ingreso, p=0.80, list = FALSE)
df_train <- df[df_index, ]
df_test <- df[-df_index, ]

In [10]:
# Preprocesamiento de los datos

set.seed(27)
preprocessData <- preProcess(df_train[,(1:24)], method=c("scale","center"))
df_train2 <- predict(preprocessData, df_train)
df_test2 <- predict(preprocessData, df_test)


In [ ]:
df_test[,25]

In [ ]:
# Entrenamiento del modelo Random Forest

rf = randomForest(ingreso ~ ., data = df_train2, mtry = 3, importance = TRUE, ntree = 500)

rf_pred = predict(rf, newdata = df_test2[,-25], type="response")

mape = mean(abs((rf_pred - df_test2$ingreso)/df_test2$ingreso))

In [ ]:
mape

In [ ]:
# Optimización del modelo Random Forest

set.seed(27)

k_folds = 5
folds = caret::createFolds(c(1:nrow(df)),k=k_folds)
m = seq(1,10, by=2)
narbol = seq(100, 1000, by = 100)
resultadosrf = data.frame()

set.seed(27)

for (k in 1:k_folds){
  for (j in m){
    for (i in narbol){
      train = df[-folds[[k]],]
      test = df[folds[[k]],]
    rf_opt = randomForest(ingreso ~ ., data = df_train2, mtry = j, importance = TRUE, ntree = i)
    rf_pred_opt = predict(rf_opt, newdata = df_test2[,-25], type="response")
    mape_opt = mean(abs((rf_pred - df_test2$ingreso)/df_test2$ingreso))
    result = c("k_folds"=k, "m"=j, "narbol"=i, "error"= mape_opt)
    resultadosrf = rbind(resultadosrf, result)
  }
  }
}


resultadosrf <- setNames(resultadosrf,c("k_folds","m","narbol","error"))
resultadosrf_cv <- aggregate(resultadosrf[,4],list(resultadosrf$m,resultadosrf$narbol),mean)
resultadosrf_cv <- setNames(resultadosrf_cv,c("m","narbol","error"))

#View(resultadosrf_cv)

resultadosrf_cv_ord <- resultadosrf_cv[order(resultadosrf_cv$error),]
head(resultadosrf_cv_ord)

In [ ]:
importance(rf)
varImpPlot(rf)

In [ ]:
# Predicción con el modelo optimizado

id <- "1dUb4neNxfSpCGRGucf_A-kcIV5t6QL1gSXDPi4uL2HA" 
df_val <- read.csv(sprintf("https://docs.google.com/uc?id=%s&export=download", id))

df_val$peorcalificacionBCU = ifelse(df_val$peorcalificacionBCU == '1C', 1,
ifelse(df_val$peorcalificacionBCU == '2A',2, ifelse(df_val$peorcalificacionBCU == '2B',3,
ifelse(df_val$peorcalificacionBCU == '3',4, ifelse(df_val$peorcalificacionBCU == '4',5,6)))))

df_val <- transform(df_val, peorcalificacionBCU = as.factor(peorcalificacionBCU))


rf_pred_def = predict(rf_final, newdata = df_val, type="response")
#View(rf_pred_def)

#df_def = data.frame(ifelse(rf_pred_def[,2]>corte,1,0))
df_def <- setNames(df_def, "prediccion")
#View(df_def)

write.csv(x = df_def, file = "df_def.csv", row.names = TRUE) 